# MLflow Interface

## Table of Contents

1. [Set Up MLflow Credential and Environment values](#Set-Up-MLflow-Credential-and-Environment-values)
2. [Get Experiments List and Details](#Get-Experiments-List-and-Details)

In [ ]:
import os
import mlflow
from mlflow.tracking import MlflowClient
import requests
import ipywidgets as widgets

from IPython.display import display

In [ ]:
domain = "ezua1.local"
mlflow_username = "chowoocheol@gmail.com"
mlflow_password = "userdemo1!"

# Set Up MLflow Credential and Environment values

Set up MLflow for model tracking and register the model. Include authentication if necessary.

In [ ]:
# Add heading
heading = widgets.HTML("<h2>MLflow Credentials</h2>")
display(heading)

domain_input = widgets.Text(description='Domain:', placeholder="ua.ezm.host")
username_input = widgets.Text(description='Username:')
password_input = widgets.Password(description='Password:')
submit_button = widgets.Button(description='Submit')
success_message = widgets.Output()

domain = None
mlflow_username = None
mlflow_password = None

def submit_button_clicked(b):
    global domain, mlflow_username, mlflow_password
    domain = domain_input.value
    mlflow_username = username_input.value
    mlflow_password = password_input.value
    with success_message:
        success_message.clear_output()
        print("Credentials submitted successfully!")
    submit_button.disabled = True

submit_button.on_click(submit_button_clicked)

# Set margin on the submit button
submit_button.layout.margin = '20px 0 20px 0'

# Display inputs and button
display(domain_input, username_input, password_input, submit_button, success_message)

In [ ]:
token_url = f"https://keycloak.{domain}/realms/UA/protocol/openid-connect/token"

data = {
    "username" : mlflow_username,
    "password" : mlflow_password,
    "grant_type" : "password",
    "client_id" : "ua-grant",
}

token_responce = requests.post(token_url, data=data, allow_redirects=True, verify=False)

token = token_responce.json()["access_token"]

In [ ]:
os.environ['MLFLOW_TRACKING_TOKEN'] = token
os.environ["AWS_ACCESS_KEY_ID"] = os.environ['MLFLOW_TRACKING_TOKEN']
os.environ["AWS_SECRET_ACCESS_KEY"] = "s3"
os.environ["AWS_ENDPOINT_URL"] = 'http://local-s3-service.ezdata-system.svc.cluster.local:30000'
os.environ["MLFLOW_S3_ENDPOINT_URL"] = os.environ["AWS_ENDPOINT_URL"]
os.environ["MLFLOW_S3_IGNORE_TLS"] = "true"
os.environ["MLFLOW_TRACKING_INSECURE_TLS"] = "true"
os.environ["MLFLOW_TRACKING_URI"] = "http://mlflow.mlflow.svc.cluster.local:5000"

# Get Experiments List and Details

get information of experemens

In [ ]:
# Function to list all experiments
def list_experiments():
    experiments = client.search_experiments()
    return experiments

# Function to get details of a specific experiment
def get_experiment_details(experiment_id):
    try:
        experiment = client.get_experiment(experiment_id)
        experiment_details = {
            "experiment_id": experiment.experiment_id,
            "name": experiment.name,
            "artifact_location": experiment.artifact_location,
            "lifecycle_stage": experiment.lifecycle_stage,
            "tags": experiment.tags,
            "last_update_time": experiment.last_update_time
        }
        
        # Retrieve runs for the specific experiment
        runs = client.search_runs(experiment_ids=[experiment_id])
        run_details = []
        for run in runs:
            run_info = {
                "run_id": run.info.run_id,
                "start_time": run.info.start_time,
                "end_time": run.info.end_time,
                "status": run.info.status,
                "artifact_uri": run.info.artifact_uri
            }
            run_details.append(run_info)
        
        experiment_details["runs"] = run_details
        return experiment_details
    
    except Exception as e:
        print(f"Error retrieving details for experiment '{experiment_id}': {e}")
        return None

In [ ]:
# List all experiments
experiments = list_experiments()
print("Experiments:")
for experiment in experiments:
    print(f"- ID: {experiment.experiment_id}, Name: {experiment.name}")
    
# Prompt the user to enter the experiment ID they want to query
experiment_id = input("Enter the experiment ID you want to query: ")

# Retrieve and display the details for the specified experiment
details = get_experiment_details(experiment_id)

if details:
    print(f"\nDetails for experiment '{experiment_id}':")
    print(details)
else:
    print(f"No details found for experiment '{experiment_id}'")